In [ ]:
#biblioteca utilizadas para suportar scripts escritos abaixo
import datetime
import re
import locale
import pyodbc 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline, BSpline
import seaborn as sns
from matplotlib.dates import DateFormatter
from matplotlib.ticker import AutoMinorLocator
import sqlalchemy as sa
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows


In [ ]:
# Para funcionar o pyodbc é necessário adicionar a seguinte linha no registro

cnxn = pyodbc.connect('Driver={SQL Server};'
                      'Server=***;'
                      'Database=***;'
                      'UID=***;'
                      'PWD=***;')
                      

In [ ]:
# Verificar a conexão
cursor = cnxn.cursor()
cursor.execute("SELECT @@version;") 
row = cursor.fetchone() 
while row: 
    print(row[0])
    row = cursor.fetchone()

In [ ]:
# Puxando os dados direto do banco do jazz atraves da query a baixo
df = pd.read_sql_query("SELECT DISTINCT T1.REFERENCE_ID, T1.CREATION_DATE, T1.RESOLVED_DATE, T2_1.LITERAL_NAME AS LITERAL_NAME, T1.REQUEST_STATE, T1.REQUEST_SEVERITY, T1.TEAM_NAME, FORMAT(CREATION_DATE,'MMM/yyyy')as CRIACAO_FORMAT, FORMAT(RESOLVED_DATE,'MMM/yyyy')as FINAL_FORMAT FROM RIDW.VW_REQUEST T1 LEFT OUTER JOIN RICALM.VW_RQST_STRING_EXT T2 ON T2.REQUEST_ID=T1.REQUEST_ID AND T2.NAME='Sistema' LEFT OUTER JOIN RICALM.REQUEST_ATTRDEF T2_2 ON T2_2.REQUEST_TYPE_ID=T1.REQUEST_TYPE_ID AND T2_2.NAME=T2.NAME LEFT OUTER JOIN RICALM.VW_RQST_ENUMERATION T2_1 ON T2_1.EXTERNAL_ID=T2.VAL AND T2_1.PROJECT_ID=T1.PROJECT_ID AND T2_1.ENUMERATION_NAME=T2_2.DATATYPE WHERE T1.PROJECT_ID = 415  AND (T1.REQUEST_TYPE = 'Incidente' AND T1.CREATION_DATE >= '2022-04-01' ) AND T1.ISSOFTDELETED = 0 AND (T1.REQUEST_ID <> -1 AND T1.REQUEST_ID IS NOT NULL)", cnxn)
df.to_excel("Dados_Revista.xlsx")

In [ ]:
#algoritimo para utilizar no filtro do mes anterior
today=datetime.date.today()
first = today.replace(day=1)
last_month = first - datetime.timedelta(days=1)
fil=(last_month.strftime("%b/%Y"))

In [ ]:
#substituo os valores na coluna dos sistemas
#df = df.replace({'LITERAL_NAME': {'(LS0073)SAP - FI-AP': '(LS0069)SAP', '(LS0078)SAP - FI-TV': '(LS0069)SAP', '(LS0077)SAP - FI-CA': '(LS0069)SAP', '(LS0075)SAP - MM': '(LS0069)SAP', '(LS0068)Interface SIAS - SAP' : '(LS0069)SAP', '(CB)SIAS - Cobrança de Prêmios' : '(LS0389)SIAS', '(VG)SIAS - Vida em Grupo' : '(LS0389)SIAS', '(VA)SIAS - Vida Azul' : '(LS0389)SIAS' , '(SI)SIAS - Sinistro':'(LS0389)SIAS', '(BI)SIAS - Bilhete' : '(LS0389)SIAS',  '(EM)SIAS - Emissão' :'(LS0389)SIAS', '(AU)SIAS - Automóvel' : '(LS0389)SIAS',  '(CO)SIAS - Comissões' :'(LS0389)SIAS', '(GV)SIAS - Geral Vida' : '(LS0389)SIAS', '(LS0209)SIAS - Vida Empresarial' : '(LS0389)SIAS', '(HB)SIAS - Habitacional' : '(LS0389)SIAS', '(GL)SIAS - Concentrador Contábil - GL' : '(LS0389)SIAS', '(MR)SIAS - Multirisco': '(LS0389)SIAS', '(LS0215)SIAS - Vida Preferencial' : '(LS0389)SIAS', '(CA)SIAS - Co-Seguro Aceita' : '(LS0389)SIAS' , '(RE)SIAS - Resseguro' : '(LS0389)SIAS', '(RG)SIAS - Registros Oficiais (Gerais)': '(LS0389)SIAS', '(LS0098b)Réplica SIAS':'(LS0389)SIAS', '(AC)SIAS - Administração de Co-seguro': '(LS0389)SIAS', '(LT)SIAS - Lotérico':'(LS0389)SIAS', '(SX)SMART Habitacional Web' : '(SX)SMART Habitacional' }})
#print(df)
#print(type(df))

In [ ]:
# Quantidade total de incidentes abertos desde 01/04/2022 // Indicador 0
quant_abertoT=df[df.columns[1]].count()

# Imprimindo o resultado
quant_abertoT

In [ ]:
df_last = df[(df["CRIACAO_FORMAT"] == last_month.strftime("%b/%Y"))]
# Agrupa as linhas da coluna [REFERENCE_ID] por mês e coluna [REQUEST_SEVERITY] utilizando a coluna [CREATION_DATE] // Indicador 2
area = df_last.groupby([df['CRIACAO_FORMAT'], 'REQUEST_SEVERITY']).count()['REFERENCE_ID']

# Imprimindo o resultado
area = area.fillna(0)
area

In [ ]:
# substitui os status detalhados por status simples para agregar os dados e posteriormente imprimir a quantidade agregada // importante para o indicador 1
df2 = df.replace({'REQUEST_STATE': {'Concluído': 'Fechado', 'Cancelado': 'Fechado', 'Aguardando Execução': 'Backlog', 'Em Execução': 'Backlog', 'Em Homologação' : 'Backlog', 'E _ Aguardando Times Internos' : 'Backlog', 'Aberto' :'Backlog', 'E _ Aguardando Demandante' : 'Backlog' , 'A _ Aguardando Demandante' : 'Backlog', 'E _ Aguardando Fornecedor':'Backlog'}}).copy()

In [ ]:
df_last = df[(df["CRIACAO_FORMAT"] == last_month.strftime("%b/%Y"))]
# Agrupa as linhas da coluna [REFERENCE_ID] por mês utilizando a coluna [CREATION_DATE] demonstrando a quantidade de incidentes abertos por mes// Indicador 1
abertoM = df_last.groupby(df['CRIACAO_FORMAT'])['REFERENCE_ID'].count()

# Imprimindo o resultado
abertoM

In [ ]:
df_last = df[(df["FINAL_FORMAT"] == last_month.strftime("%b/%Y"))]
# Agrupa as linhas da coluna [REFERENCE_ID] por mês utilizando a coluna [CREATION_DATE] demonstrando a quantidade de incidentes abertos por mes// Indicador 1
fechadoM = df_last.groupby(df['FINAL_FORMAT'])['REFERENCE_ID'].count()

# Imprimindo o resultado
fechadoM

In [ ]:
# Agrupa as linhas da coluna [REFERENCE_ID] por backlog/ Indicador 1
opcl=df2
Backlog = opcl[opcl['REQUEST_STATE'].isin(['Backlog'])]

Backlog1 = Backlog.groupby(['REQUEST_STATE']).count()['REFERENCE_ID']

# Imprimindo o resultado
Backlog1

In [ ]:
df_last1 = df[(df["FINAL_FORMAT"] == last_month.strftime("%b/%Y"))]
#filtra as linhas que tem o status cancelados
cancelados = df_last1[df_last1['REQUEST_STATE'].isin(['Cancelado'])]

#imprime o agrupamento das linhas da coluna [REFERENCE_ID] por mes utilizando a coluna [FINAL_FORMAT] demonstrando a quantidade de incidentes cancelados // Indicador 3
cancelados = cancelados.groupby([df_last1['FINAL_FORMAT'], 'REQUEST_STATE']).count()['REFERENCE_ID']

cancelados

In [ ]:


df_last = df
sis_ab =  df_last[(df_last["LITERAL_NAME"] == "(LS0069)SAP") | (df_last["LITERAL_NAME"] == "(SZ)SMART Prestamista") | (df_last["LITERAL_NAME"] == "(LS0270)Salesforce") | (df_last["LITERAL_NAME"] == "(LS0305)I4Pro (Previsul)") | (df_last["LITERAL_NAME"] == "(LS0105)Soluções de Infraestrutura (Aplicações de Mercado)") | (df_last["LITERAL_NAME"] == "(LS0064)NEWCON") | (df_last["LITERAL_NAME"] == "(LS0389)SIAS") | (df_last["LITERAL_NAME"] == "(LS0289)CCM 2.0 - Inspire Quadient") | (df_last["LITERAL_NAME"] == "(LS0671)IBM Sterling B2B Integrator")]
# Quantidade total de incidentes abertos desde 01/04/2022 // Indicador 4
quant_abertoT=sis_ab[sis_ab.columns[1]].count()
# Imprimindo o resultado
quant_abertoT

In [ ]:
df_last = df[(df["CRIACAO_FORMAT"] == last_month.strftime("%b/%Y"))]
# Selecionando as linhas em que a coluna 'LITERAL_NAME' é igual aos sistemas no indicador 4 // Indicador 4
sis_ab =  df_last[(df_last["LITERAL_NAME"] == "(LS0069)SAP") | (df_last["LITERAL_NAME"] == "(SZ)SMART Prestamista") | (df_last["LITERAL_NAME"] == "(LS0270)Salesforce") | (df_last["LITERAL_NAME"] == "(LS0305)I4Pro (Previsul)") | (df_last["LITERAL_NAME"] == "(LS0105)Soluções de Infraestrutura (Aplicações de Mercado)") | (df_last["LITERAL_NAME"] == "(LS0064)NEWCON") | (df_last["LITERAL_NAME"] == "(LS0389)SIAS") | (df_last["LITERAL_NAME"] == "(LS0289)CCM 2.0 - Inspire Quadient") | (df_last["LITERAL_NAME"] == "(LS0671)IBM Sterling B2B Integrator")]

# Agrupando as linhas selecionadas por sistema e contando a quantidade de requisições abertas para cada sistema por mes
sis_ab = sis_ab.groupby(['LITERAL_NAME', 'CRIACAO_FORMAT']).count()['REFERENCE_ID']

# Imprime o resultado de forma ascendente(do maior para o menor)
sis_ab#.sort_values(ascending=False)

In [ ]:
df_last1 = df2
sis_fc =  df_last1[(df_last1["LITERAL_NAME"] == "(LS0069)SAP") | (df_last1["LITERAL_NAME"] == "(SZ)SMART Prestamista") | (df_last1["LITERAL_NAME"] == "(LS0270)Salesforce") | (df_last1["LITERAL_NAME"] == "(LS0305)I4Pro (Previsul)") | (df_last1["LITERAL_NAME"] == "(LS0105)Soluções de Infraestrutura (Aplicações de Mercado)") | (df_last1["LITERAL_NAME"] == "(LS0064)NEWCON") | (df_last1["LITERAL_NAME"] == "(LS0389)SIAS") | (df_last1["LITERAL_NAME"] == "(LS0289)CCM 2.0 - Inspire Quadient") | (df_last1["LITERAL_NAME"] == "(LS0671)IBM Sterling B2B Integrator")]
sis_fc = df_last1[df_last1['REQUEST_STATE'].isin(['Fechado'])]
# Agrupando as linhas selecionadas por sistema e contando a quantidade de requisições fechadas para cada sistema por mes
sis_fc = sis_fc.groupby(['REQUEST_STATE']).count()['REFERENCE_ID']

sis_fc

In [ ]:
df_last1 = df[(df["FINAL_FORMAT"] == last_month.strftime("%b/%Y"))]
# Selecionando as linhas em que a coluna 'LITERAL_NAME' é igual aos sistemas no indicador 5 // Indicador 5
sis_fc =  df_last1[(df_last1["LITERAL_NAME"] == "(LS0069)SAP") | (df_last1["LITERAL_NAME"] == "(SZ)SMART Prestamista") | (df_last1["LITERAL_NAME"] == "(LS0270)Salesforce") | (df_last1["LITERAL_NAME"] == "(LS0305)I4Pro (Previsul)") | (df_last1["LITERAL_NAME"] == "(LS0105)Soluções de Infraestrutura (Aplicações de Mercado)") | (df_last1["LITERAL_NAME"] == "(LS0064)NEWCON") | (df_last1["LITERAL_NAME"] == "(LS0389)SIAS") | (df_last1["LITERAL_NAME"] == "(LS0289)CCM 2.0 - Inspire Quadient") | (df_last1["LITERAL_NAME"] == "(LS0671)IBM Sterling B2B Integrator")]

# Agrupando as linhas selecionadas por sistema e contando a quantidade de requisições fechadas para cada sistema por mes
sis_fc = sis_fc.groupby(['LITERAL_NAME', 'FINAL_FORMAT']).count()['REFERENCE_ID']

sis_fc

In [ ]:
#algoritimo para utilizar no filtro do mes anterior
today=datetime.date.today()
first = today.replace(day=1)
last_month = first - datetime.timedelta(days=1)
fil=(last_month.strftime("%b/%Y"))

In [ ]:
#algoritimo para utilizar no filtro do mes atual
#today=datetime.date.today()
#first = today
#last_month = first 
#fil=(last_month.strftime("%b/%Y"))

In [ ]:

df_last = df[(df["CRIACAO_FORMAT"] == last_month.strftime("%b/%Y"))]
# Agrupa as linhas da coluna [REFERENCE_ID] por mês e coluna [LITERAL_NAME] utilizando a coluna [CREATION_DATE]
sis = df_last.groupby([df_last['CRIACAO_FORMAT'], 'LITERAL_NAME']).count()['REFERENCE_ID']

# Ordena o resultado de forma descendente
sis = sis.sort_values(ascending=False)

# Pega os 3 principais sistemas
top3 = sis.nlargest(6)

# Pega os demais sistemas
others = sis.nsmallest(len(sis)-6)

# Some os demais sistemas
others_sum = others.sum()
# Adiciona os demais sistemas ao top3
top31 = top3.append(pd.Series({"Outros": others_sum}, name='Outros'))

top31


In [ ]:

dft3 = df[(df["CRIACAO_FORMAT"] == last_month.strftime("%b/%Y"))]
# Agrupa as linhas da coluna [REFERENCE_ID] por mês e coluna [LITERAL_NAME] utilizando a coluna [CREATION_DATE]
sis = dft3.groupby([dft3['CRIACAO_FORMAT'], 'TEAM_NAME']).count()['REFERENCE_ID']

# Ordena o resultado de forma descendente
sis = sis.sort_values(ascending=False)

# Pega os 3 principais sistemas
top3 = sis.nlargest(6)

# Pega os demais sistemas
others = sis.nsmallest(len(sis)-6)

# Some os demais sistemas
others_sum = others.sum()
# Adiciona os demais sistemas ao top3
top32 = top3.append(pd.Series({"Outros": others_sum}, name='Outros'))

top32


In [ ]:

dft2 = df[(df["FINAL_FORMAT"] == last_month.strftime("%b/%Y"))]
# Agrupa as linhas da coluna [REFERENCE_ID] por mês e coluna [LITERAL_NAME] utilizando a coluna [FINAL_FORMAT]
sis = dft2.groupby([dft2['FINAL_FORMAT'], 'LITERAL_NAME']).count()['REFERENCE_ID']

# Ordena o resultado de forma descendente
sis = sis.sort_values(ascending=False)

# Pega os 3 principais sistemas
top3 = sis.nlargest(6)

# Pega os demais sistemas
others = sis.nsmallest(len(sis)-6)

# Some os demais sistemas
others_sum = others.sum()
# Adiciona os demais sistemas ao top3
top33 = top3.append(pd.Series({"Outros": others_sum}, name='Outros'))

top33


In [ ]:
dft = df[(df["FINAL_FORMAT"] == last_month.strftime("%b/%Y"))]
# Agrupa as linhas da coluna [REFERENCE_ID] por mês e coluna [LITERAL_NAME] utilizando a coluna [CREATION_DATE]
sis = dft.groupby([dft['FINAL_FORMAT'], 'TEAM_NAME']).count()['REFERENCE_ID']

# Ordena o resultado de forma descendente
sis = sis.sort_values(ascending=False)

# Pega os 3 principais sistemas
top3 = sis.nlargest(6)

# Pega os demais sistemas
others = sis.nsmallest(len(sis)-6)

# Some os demais sistemas
others_sum = others.sum()
# Adiciona os demais sistemas ao top3
top34 = top3.append(pd.Series({"Outros": others_sum}, name='Outros'))

top34


In [ ]:
# Selecionando as linhas em que a coluna 'REQUEST_STATE' é diferente de concluido e cancelado demonstrando os incidentes que ainda estão abertos // Indicador 10
backlog_st = df2[(df2["REQUEST_STATE"] == "Backlog")]

# Agrupando as linhas selecionadas por sistema e contando a quantidade de requisições para cada sistema
backlog_st = backlog_st.groupby('TEAM_NAME')['REFERENCE_ID'].count()

# Imprime o resultado de forma ascendente(do maior para o menor)
backlog_st.sort_values(ascending=False)


# Pega os 3 principais sistemas
top3 = backlog_st.nlargest(6)

# Pega os demais sistemas
others = backlog_st.nsmallest(len(backlog_st)-6)

# Some os demais sistemas
others_sum = others.sum()
# Adiciona os demais sistemas ao top3
top35 = top3.append(pd.Series({"Outros": others_sum}, name='Outros'))

top35



In [ ]:
# Selecionando as linhas em que a coluna 'REQUEST_STATE' é diferente de concluido e cancelado demonstrando os incidentes que ainda estão abertos // Indicador 11
sisbacklog = df2[(df2["REQUEST_STATE"] == "Backlog")]

# Agrupando as linhas selecionadas por sistema e contando a quantidade de requisições para cada sistema
sisbacklog = sisbacklog.groupby('LITERAL_NAME')['REFERENCE_ID'].count()

# Imprime o resultado de forma ascendente(do maior para o menor)
sisbacklog.sort_values(ascending=False)

# Ordena o resultado de forma descendente
sisbacklog = sisbacklog.sort_values(ascending=False)

# Pega os 3 principais sistemas
top3 = sisbacklog.nlargest(6)

# Pega os demais sistemas
others = sisbacklog.nsmallest(len(sis)-6)

# Some os demais sistemas
others_sum = others.sum()
# Adiciona os demais sistemas ao top3
top36 = top3.append(pd.Series({"Outros": others_sum}, name='Outros'))

top36


In [ ]:
# Selecionando as linhas em que a coluna 'REQUEST_STATE' é diferente de concluido e cancelado demonstrando os incidentes que ainda estão abertos // Indicador 12
backlog_st = df[(df["REQUEST_STATE"] != "Concluído") & (df["REQUEST_STATE"] != "Cancelado")]
backlog_st = backlog_st.replace({'REQUEST_STATE': {'Aguardando Execução': 'Aguardando Execução', 'Em Execução': 'Em Execução', 'Em Homologação' : 'Em Execução', 'E _ Aguardando Times Internos' : 'Pendentes Outros', 'Aberto' :'Em Execução', 'E _ Aguardando Demandante' : 'Pendentes Outros' , 'E _ Aguardando Fornecedor':'Pendentes Outros'}})

# Agrupando as linhas selecionadas por status e contando a quantidade de requisições
backlog_st0 = backlog_st.groupby('REQUEST_STATE')['REFERENCE_ID'].count()

# Imprime o resultado de forma ascendente(do maior para o menor)
backlog_st0

In [ ]:
# Selecionando as linhas em que a coluna 'REQUEST_STATE' é diferente de concluido e cancelado demonstrando os incidentes que ainda estão abertos // Indicador 13
backlog_st1 = df[(df["REQUEST_STATE"] != "Concluído") & (df["REQUEST_STATE"] != "Cancelado")]


# Agrupando as linhas selecionadas por status e contando a quantidade de requisições
backlog_st1 = backlog_st1.groupby('REQUEST_STATE')['REFERENCE_ID'].count()

# Imprime o resultado de forma ascendente(do maior para o menor)
print(backlog_st1.sort_values(ascending=False))

In [ ]:
filename = 'incidentes.xlsx'
writer = pd.ExcelWriter(filename, engine='xlsxwriter')

# In[3]:
wb = Workbook()
ws = wb.active
# Cria DataFrame para o valor de quant_abertoT
quant_abertoT_df = pd.DataFrame({'Total de incidentes abertos': [quant_abertoT]})
# Salva o DataFrame em uma planilha no arquivo Excel
quant_abertoT_df.to_excel(writer, sheet_name='Total de incidentes abertos')
area.to_excel(writer, sheet_name='abertos severidade no mes')
abertoM.to_excel(writer, sheet_name='bertos no mes')
Backlog1.to_excel(writer, sheet_name='backlog no mes')
cancelados.to_excel(writer, sheet_name='cancelados no mes')
sis_ab.to_excel(writer, sheet_name='Dados detl mes sistemas AB')
sis_fc.to_excel(writer, sheet_name='Dados detl mes sistemas FC')
top31.to_excel(writer, sheet_name='Aberto -  MEs Sistema')
top32.to_excel(writer, sheet_name='Aberto -  MEs Gerencia')
top33.to_excel(writer, sheet_name='Fechado - MEs Sistema')
top34.to_excel(writer, sheet_name='Fechado - MEs Gerencia')
top35.to_excel(writer, sheet_name='Blg Acumulado - MEs Gerencia')
top36.to_excel(writer, sheet_name='Blg Acumulado - MEs Sistema')
backlog_st0.to_excel(writer, sheet_name='Blg Acumulado - MEs Status agrp')
backlog_st1.to_excel(writer, sheet_name='Blg Acumulado - MEs Status detl')

writer.save()
print(f"Os dados foram salvos em {filename} com sucesso!")